In [11]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN


In [12]:

def load_and_extract_data(json_file):
    # Cargar JSON
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    records = []
    for record in data:
        # Extraer datos básicos
        declaracion = record.get('declaracion', {}).get('situacionPatrimonial', {})
        datos_generales = declaracion.get('datosGenerales', {})

        # Datos del funcionario
        nombre = f"{datos_generales.get('nombre', '')} {datos_generales.get('primerApellido', '')}"

        # Extraer escolaridad
        educacion = declaracion.get('datosCurricularesDeclarante', {}).get('escolaridad', [])
        for edu in educacion:
            records.append({
                'id': record.get('id'),
                'nombre': nombre,
                'tipo': 'educacion',
                'texto': edu.get('carreraAreaConocimiento', ''),
                'nivel': edu.get('nivel', {}).get('valor', ''),
                'institucion': edu.get('institucionEducativa', {}).get('nombre', '')
            })

        # Extraer función principal
        empleo = declaracion.get('datosEmpleoCargoComision', {})
        records.append({
            'id': record.get('id'),
            'nombre': nombre,
            'tipo': 'funcion',
            'texto': empleo.get('funcionPrincipal', ''),
            'cargo': empleo.get('empleoCargoComision', ''),
            'nivel': empleo.get('nivelEmpleoCargoComision', '')
        })

    return pd.DataFrame(records)

def aplicar_reglas_directas(texto, tipo):
    """Aplicar reglas básicas de validación"""
    anomalias = []

    # Reglas generales
    if not texto or texto.isspace():
        anomalias.append("Campo vacío o solo espacios")
        return anomalias

    if len(texto) < 5:
        anomalias.append("Texto demasiado corto")

    if texto.isupper():
        anomalias.append("Texto completamente en mayúsculas")

    # Reglas específicas para educación
    if tipo == 'educacion':
        palabras_genericas = ['ADMINISTRATIVA', 'OTRO', 'GENERAL', 'ESPECIALIDAD']
        if any(p in texto.upper() for p in palabras_genericas):
            anomalias.append("Uso de términos demasiado genéricos")

        if any(char.isdigit() for char in texto):
            anomalias.append("Contiene números en nombre de carrera")

    # Reglas específicas para función
    if tipo == 'funcion':
        if 'otro' in texto.lower() and len(texto) < 20:
            anomalias.append("Uso de 'Otro' sin especificación detallada")

        palabras_esperadas = ['administración', 'gestión', 'supervisión', 'coordinación']
        if not any(p in texto.lower() for p in palabras_esperadas):
            anomalias.append("No contiene palabras típicas de función")

    return anomalias

def detectar_anomalias_distribucion(df, tipo_campo):
    """Detectar anomalías usando TF-IDF y clustering"""
    subset = df[df['tipo'] == tipo_campo]
    textos = subset['texto'].tolist()

    # Crear vectores TF-IDF
    vectorizer = TfidfVectorizer(min_df=1, max_df=0.9)
    tfidf_matrix = vectorizer.fit_transform(textos)

    # Aplicar DBSCAN para clustering
    clustering = DBSCAN(eps=0.5, min_samples=2)
    clusters = clustering.fit_predict(tfidf_matrix.toarray())

    # Identificar outliers (etiquetados como -1 por DBSCAN)
    anomalias = []
    for idx, (_, row) in enumerate(subset.iterrows()):
        if clusters[idx] == -1:
            anomalias.append({
                'id': row['id'],
                'nombre': row['nombre'],
                'texto': row['texto'],
                'razon': "Patrón de texto inusual comparado con otros registros"
            })

    return anomalias

def analizar_declaraciones(json_file):
    # Cargar y preparar datos
    df = load_and_extract_data(json_file)

    resultados = []

    # Analizar cada registro
    for _, row in df.iterrows():
        # Aplicar reglas directas
        anomalias = aplicar_reglas_directas(row['texto'], row['tipo'])

        if anomalias:
            resultados.append({
                'tipo_analisis': 'reglas_directas',
                'id': row['id'],
                'nombre': row['nombre'],
                'tipo_campo': row['tipo'],
                'texto': row['texto'],
                'anomalias': anomalias,
                'contexto': f"Nivel: {row['nivel']}, " +
                          (f"Institución: {row['institucion']}" if row['tipo'] == 'educacion'
                           else f"Cargo: {row['cargo']}")
            })

    # Analizar distribución de texto
    for tipo in ['educacion', 'funcion']:
        anomalias_dist = detectar_anomalias_distribucion(df, tipo)
        for anom in anomalias_dist:
            resultados.append({
                'tipo_analisis': 'distribucion',
                'id': anom['id'],
                'nombre': anom['nombre'],
                'tipo_campo': tipo,
                'texto': anom['texto'],
                'anomalias': [anom['razon']]
            })

    return resultados

# Ejecutar análisis
def generar_reporte(json_file):
    resultados = analizar_declaraciones(json_file)

    print("# Reporte de Anomalías en Declaraciones\n")

    # Agrupar por tipo de campo
    for tipo in ['educacion', 'funcion']:
        print(f"\n## Anomalías en {tipo.title()}\n")

        anomalias_tipo = [r for r in resultados if r['tipo_campo'] == tipo]
        for i, anomalia in enumerate(anomalias_tipo, 1):
            print(f"### {i}. Registro Anómalo")
            print(f"- **Declarante:** {anomalia['nombre']}")
            print(f"- **Texto:** {anomalia['texto']}")
            print("- **Problemas detectados:**")
            for a in anomalia['anomalias']:
                print(f"  - {a}")
            if 'contexto' in anomalia:
                print(f"- **Contexto:** {anomalia['contexto']}")
            print()


In [13]:
# Uso
json_file = 'PDN_S1_Guanajuato_data-0000000094.json'
generar_reporte(json_file)

# Reporte de Anomalías en Declaraciones


## Anomalías en Educacion

### 1. Registro Anómalo
- **Declarante:** MARTHA SUAREZ
- **Texto:** CONTADURIA
- **Problemas detectados:**
  - Texto completamente en mayúsculas
- **Contexto:** Nivel: LICENCIATURA, Institución: Colegio de Dirección Empresarial y Negocios

### 2. Registro Anómalo
- **Declarante:** JANET CORNEJO
- **Texto:** ADMINISTRATIVA
- **Problemas detectados:**
  - Texto completamente en mayúsculas
  - Uso de términos demasiado genéricos
- **Contexto:** Nivel: LICENCIATURA, Institución: INSTITUTO DE JUSTICIA ADMINISTRATIVA DEL ESTADO DE GUANAJUATO

### 3. Registro Anómalo
- **Declarante:** GERARDO GUERRERO
- **Texto:** LICENCIADO EN DERECHO
- **Problemas detectados:**
  - Texto completamente en mayúsculas
- **Contexto:** Nivel: LICENCIATURA, Institución: UNIVERSIDAD DE GUANAJUATO

### 4. Registro Anómalo
- **Declarante:** RICARDO RAMIREZ
- **Texto:** ESPECIALIDAD EN JUSTICIA ADMINISTRATIVA
- **Problemas detectados:**
  - Texto c